In [2]:
%load_ext autoreload
%autoreload 2
import time, base64,ssl, os, re,json5, pprint,random , math, hashlib, inspect,requests
from selenium import webdriver# Controls the browser
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By                   # Locators (ID, CLASS_NAME, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait       # Waits for elements to appear
from selenium.webdriver.support import expected_conditions as EC  # Conditions like "visible", "clickable"
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from app.utils import Helper
from app.operation_executor import OperationExecutor
import pandas as pd
import numpy as np
from io import StringIO

In [16]:
path = r"C:\Users\kaustubh.keny\Projects\JSON25\scrape_output\cache\2025-09-09\25-09-09T15-14-38_cache_PVB.json"
data = Helper.load_json(path)
ops = OperationExecutor()

ops.generate_sorted_doc_report(data)

'DepositRate_Comparison_Report.docx'

In [ ]:
driver = webdriver.Chrome()
driver.get("https://www.amfiindia.com/research-information/other-data/mf-scheme-performance-details")
wait = WebDriverWait(driver, 10)

iframe = wait.until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))
driver.switch_to.frame(iframe)

dropdown_1 = ['Open Ended', 'Close Ended']
dropdown_2 = ['Equity', 'Debt', 'Hybrid', 'Solution Oriented', 'Other']

def select_dropdown(dropdown_element, value):
    arrow = dropdown_element.find_element(By.CSS_SELECTOR, ".ng-arrow-wrapper")
    arrow.click()
    time.sleep(0.5)
    options = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".ng-option")))
    for opt in options:
        if opt.text.strip() == value:
            opt.click()
            break
    time.sleep(0.5)

# Only 3 options printed for nowwww
for op1 in dropdown_1:
    dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")
    select_dropdown(dropdowns[0], op1)

    for op2 in dropdown_2:
        dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")
        select_dropdown(dropdowns[1], op2)

        dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")
        arrow = dropdowns[2].find_element(By.CSS_SELECTOR, ".ng-arrow-wrapper")
        arrow.click()
        time.sleep(0.5)

        options = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".ng-option")))
        op3_options = [opt.text.strip() for opt in options]

        arrow.click()

        print(f"{op1} + {op2} → {op3_options}")

driver.quit()


In [18]:
dropdown_1 = ['Open Ended', 'Close Ended']
op3_map = {
    "Equity": ['Large Cap', 'Large & Mid Cap', 'Flexi Cap', 'Multi Cap', 'Mid Cap',
               'Small Cap', 'Value', 'ELSS', 'Contra', 'Dividend Yield', 'Focused', 'Sectoral / Thematic'],
    "Debt": ['Long Duration', 'Medium to Long Duration', 'Short Duration', 'Medium Duration',
             'Money Market', 'Low Duration', 'Ultra Short Duration', 'Liquid', 'Overnight',
             'Dynamic Bond', 'Corporate Bond', 'Credit Risk', 'Banking and PSU', 'Floater',
             'FMP', 'Gilt', 'Gilt with 10 year constant duration'],
    "Hybrid": ['Aggressive Hybrid', 'Conservative Hyrbid', 'Equity Savings', 'Arbitrage',
               'Multi Asset Allocation', 'Dynamic Asset Allocation or Balanced Advantage', 'Balanced Hybrid'],
    "Solution Oriented": ["Children's", 'Retirement'],
    "Other": ['Index Funds ETFs', 'FoFs (Overseas/Domestic)']
}

dropdown_4_value = "All"

all_combos = []
for op1 in dropdown_1:
    for op2, op3_list in op3_map.items():
        for op3 in op3_list:
            all_combos.append({
                "op1": op1,
                "op2": op2,
                "op3": op3,
                "op4": dropdown_4_value
            })

# print(f"Total combinations: {len(all_combos)}")
# all_combos

In [19]:
import os, time, json, shutil, glob
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
import pandas as pd

DATE_STR = "30-Jul-2025"
DOWNLOAD_DIR = os.path.join(os.getcwd(), f"mf_downloads_{DATE_STR}")
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 10)

driver.get("https://www.amfiindia.com/research-information/other-data/mf-scheme-performance-details")
iframe = wait.until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))
driver.switch_to.frame(iframe)

# Small test set
# combos = [
#     {"op1": "Open Ended", "op2": "Debt", "op3": "Overnight", "op4": "All"},
#     {"op1": "Open Ended", "op2": "Debt", "op3": "Liquid", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Large Cap", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Large & Mid Cap", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Flexi Cap", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Multi Cap", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Mid Cap", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Small Cap", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Value", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "ELSS", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Contra", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Dividend Yield", "op4": "All"},
# ]

combos = all_combos

def select_dropdown(index, value, retries=3):
    for attempt in range(retries):
        try:
            dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")
            dropdown = dropdowns[index]
            arrow = dropdown.find_element(By.CSS_SELECTOR, ".ng-arrow-wrapper")
            arrow.click()
            time.sleep(0.5)

            options = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".ng-option")))
            for opt in options:
                if opt.text.strip() == value:
                    driver.execute_script("arguments[0].scrollIntoView(true);", opt)
                    opt.click()
                    time.sleep(0.5)
                    return
            raise ValueError(f"Option '{value}' not found")
        except Exception as e:
            print(f"Retrying select for '{value}' ({attempt+1}/{retries}) due to {e.__class__.__name__}")
            time.sleep(1)
    raise Exception(f"Failed to select '{value}' after retries")

def set_date(date_str):
    date_input = driver.find_element(By.CSS_SELECTOR, "input[bsdatepicker]")
    driver.execute_script("""
        arguments[0].value = arguments[1];
        arguments[0].dispatchEvent(new Event('input', { bubbles: true }));
        arguments[0].dispatchEvent(new Event('change', { bubbles: true }));
    """, date_input, date_str)
    time.sleep(1)

def click_go():
    go_button = driver.find_element(By.CSS_SELECTOR, "button[mat-raised-button]")
    go_button.click()
    time.sleep(2)

def download_excel():
    excel_icon = driver.find_element(By.CSS_SELECTOR, ".excel-border.cursor-pointer")
    excel_icon.click()
    time.sleep(1)

def if_data_avalaible():
    try:
        snackbars = driver.find_elements(By.CSS_SELECTOR, "snack-bar-container")
        for snackbar in snackbars:
            if "Data Not Available" in snackbar.text:
                close_icon = snackbar.find_element(By.XPATH, ".//mat-icon[contains(text(), 'close')]")
                close_icon.click()
                time.sleep(1)
                print("Data Not Available. Snackbar closed.")
                return False
    except:
        pass
    return True

def modify_excel_data(combo):
    time.sleep(2)
    files = glob.glob(os.path.join(DOWNLOAD_DIR, "*.xlsx"))
    latest_file = max(files, key=os.path.getctime)

    original_name = os.path.basename(latest_file).replace(".xlsx", "")
    combo_str = "_".join([combo["op1"], combo["op2"], combo["op3"], combo["op4"]]).replace(" ", "-")
    new_name = f"Fund-Performance-{DATE_STR}-{combo_str}.xlsx"
    new_path = os.path.join(DOWNLOAD_DIR, new_name)

    counter = 1
    while os.path.exists(new_path):
        new_name = f"{original_name}_{combo_str}({counter}).xlsx"
        new_path = os.path.join(DOWNLOAD_DIR, new_name)
        counter += 1

    os.rename(latest_file, new_path)

    df_full = pd.read_excel(new_path, header=None)
    df_top = df_full.iloc[:4]
    df_data = df_full.iloc[4:].copy()

    for idx, val in enumerate(combo.values()):
        df_data.insert(loc=idx, column=f'Option_{idx+1}', value=val)

    for idx in range(len(combo)):
        df_top.insert(loc=idx, column=f'Option_{idx+1}', value=[""] * len(df_top))

    df_final = pd.concat([df_top, df_data], ignore_index=True)

    mod_dir = os.path.join(DOWNLOAD_DIR, "mod")
    os.makedirs(mod_dir, exist_ok=True)
    mod_path = os.path.join(mod_dir, new_name)

    df_final.to_excel(mod_path, index=False, header=False)
    print(f"Modified file saved to: {mod_path}")

not_avalaible_combos = []

for combo in combos:
    try:
        op1, op2, op3, op4 = combo["op1"], combo["op2"], combo["op3"], combo["op4"]

        print(f"\nPreparing combo: {op1}, {op2}, {op3}, {op4}")
    
        set_date(DATE_STR)
        select_dropdown(0, op1)
        select_dropdown(1, op2)
        select_dropdown(2, op3)
        select_dropdown(3, op4)

        click_go()
        time.sleep(2)
        if not if_data_avalaible():
            not_avalaible_combos.append(combos)
            continue
                
        download_excel()
        print("Download triggered — waiting for file...")
        modify_excel_data(combo)
        time.sleep(2)

    except Exception as e:
        print(f"[ERROR] {combo}: {e}")

driver.quit()

with open("not_open.json","w+") as f:
    json.dump(not_avalaible_combos,f)


Preparing combo: Open Ended, Equity, Large Cap, All
Download triggered — waiting for file...
Modified file saved to: c:\Users\kaustubh.keny\Projects\office-work\selenium-scrape\mf_downloads_30-Jul-2025\mod\Fund-Performance-30-Jul-2025-Open-Ended_Equity_Large-Cap_All.xlsx

Preparing combo: Open Ended, Equity, Large & Mid Cap, All
Download triggered — waiting for file...
Modified file saved to: c:\Users\kaustubh.keny\Projects\office-work\selenium-scrape\mf_downloads_30-Jul-2025\mod\Fund-Performance-30-Jul-2025-Open-Ended_Equity_Large-&-Mid-Cap_All.xlsx

Preparing combo: Open Ended, Equity, Flexi Cap, All
Download triggered — waiting for file...
Modified file saved to: c:\Users\kaustubh.keny\Projects\office-work\selenium-scrape\mf_downloads_30-Jul-2025\mod\Fund-Performance-30-Jul-2025-Open-Ended_Equity_Flexi-Cap_All.xlsx

Preparing combo: Open Ended, Equity, Multi Cap, All
Download triggered — waiting for file...
Modified file saved to: c:\Users\kaustubh.keny\Projects\office-work\seleniu

In [ ]:
def create_grand_sheet(path:str):
    all_dataframes = []
    excel_files = sorted(glob.glob(os.path.join(path, "*.xlsx")))
    
    all_data_path = os.path.join(path, "ALL_DATA_FILE.xlsx")
    grand_path = os.path.join(path, f"Grand_Stacked_{DATE_STR}.xlsx")
    
    #stack
    for file in excel_files:
        df = pd.read_excel(file, header=None)
        empty_rows = pd.DataFrame([[""] * df.shape[1]] * 3)
        all_dataframes.append(df)
        all_dataframes.append(empty_rows)

    grand_df = pd.concat(all_dataframes, ignore_index=True)
    grand_df.to_excel(grand_path, index=False, header=False)
    
    #one shot
    with pd.ExcelWriter(all_data_path, engine='openpyxl') as writer:
        for i, file in enumerate(excel_files):
            sheet_name = f"Sheet{i+1}"
            df = pd.read_excel(file, header=None)
            df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

    print(f" `ALL_DATA_FILE` created with {len(excel_files)} sheets: {all_data_path}")
    print(f"Grand Excel created with {len(excel_files)} : {grand_path}")


path = r"C:\Users\kaustubh.keny\Projects\office-work\selenium-scrape\mf_downloads_30-Jul-2025\mod"

create_grand_sheet(path)


In [2]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
from urllib.parse import urljoin
import os
ops = OperationExecutor()
path = r"25-09-07T23-17-03_cache.json5"
# path = r"25-09-08T18-33-09_cache.json5"

    
root_file = os.path.basename(path).split("_")[0]
function_to_execute = {
"primary":[["normalize_df","value","norm_table","table_html"],["sha1","value","SHA_ONE","pdf"]],
"secondary":[["sha1","norm_table","SHA_ONE"]],
}
try:
    data = Helper.load_json(path, typ="json5")
    processed_data = ops.runner(data,function_to_execute)
    Helper.save_json(processed_data,f"{root_file}_process.json")
except Exception as e:
    print(f"\nError in Processing.. Skipping, {e}")


>>Processing Bank of Baroda
>>Processing Bank of India
>>Processing Bank of Maharashtra
>>Processing Canara Bank
>>Processing Central Bank of India
>>Processing Indian Bank
>>Processing Indian Overseas Bank
>>Processing Punjab and Sind Bank
>>Processing Punjab National Bank
>>Processing State Bank of India
>>Processing Uco Bank
>>Processing Union Bank of India
>>Processing Axis Bank
>>Processing Bandhan Bank
>>Processing CSB Bank
>>Processing City Union Bank
>>Processing DCB Bank
>>Processing Dhanlaxmi Bank
>>Processing Federal Bank
>>Processing HDFC Bank
>>Processing ICICI Bank
>>Processing IDBI Bank
>>Processing IDFC First Bank
>>Processing IndusInd Bank
>>Processing Jammu and Kashmir Bank
>>Processing Karnataka Bank
>>Processing Karur Vysya Bank
>>Processing Kotak Mahindra Bank
>>Processing The Nainital Bank
>>Processing South Indian Bank
>>Processing Tamilnad Mercantile Bank
>>Processing RBL Bank


In [3]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
ops = OperationExecutor()
yesterday = Helper.load_json("25-09-07T23-17-03_process.json")
today = Helper.load_json("25-09-08T18-33-09_process.json")

result = ops.process_comparison(yesterday,today,key="SHA_ONE")
Helper.save_json(result, "compare-080925.json")

ops.generate_sorted_excel_report(result)

'DepositRate_Comparison_Report.xlsx'